In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os
import argparse
import sys

In [2]:
if '-f' in sys.argv:
    sys.argv.remove('-f')
    sys.argv.remove('C:\\Users\\Fran\\AppData\\Roaming\\jupyter\\runtime\\kernel-f7be5c68-6126-4bee-ba62-4de6c32df41d.json')


# Create ArgumentParser object
args = argparse.ArgumentParser(allow_abbrev=False)

# Add arguments
args.add_argument("--mode", default='webcam')
args.add_argument("--filePath", default=None)

# Parse arguments
args = args.parse_args()

In [3]:
#img_path = 'data/img.png'
output_dir = 'data/'
#img = cv2.imread(img_path)
#cv2.imshow(img)
#H, W ,_ = img.shape

In [4]:
def process_img(img, face_detection):
    H, W ,_ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out = face_detection.process(img)
    
    if out.detections is not None:
        for detection in out.detections:  # Note the corrected attribute name 'detections'
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box

            x1, y1, w, h = bbox.xmin, bbox.ymin, bbox.width, bbox.height
            x1 = int(x1*W)
            y1 = int(y1*H)
            w = int(w*W)
            h = int(h*H)
            
            #cv2.rectangle(img, (x1,y1), (x1+w, y1+h), (0,255,0), 10)
            
            #blur the face only
            img[y1:y1 + h, x1:x1 + w, :] = cv2.blur(img[y1:y1 + h, x1:x1 + w, :], (25,25))
    return img

In [5]:
mp_face_detection = mp.solutions.face_detection
#model selection 0 to near objects and 1 to far
with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence = 0.5) as face_detection:
   

    if args.mode in ["image"]:
        img = cv2.imread(args.filePath)
        img = process_img(img, face_detection)
        cv2.imwrite(os.path.join(output_dir, 'ou42133.png'), img)
    elif args.mode in ["video"]:
        cap = cv2.VideoCapture(args.filePath)
        ret, frame = cap.read()
        output_video = cv2.VideoWriter(os.path.join(output_dir, 'videooo2.mp4'),
                                      cv2.VideoWriter_fourcc(*'MP4V'),
                                      25, 
                                      (frame.shape[1], frame.shape[0]))
        while ret:
            frame = process_img(frame, face_detection)
            output_video.write(frame)
            ret, frame = cap.read()
        cap.release()
        output_video.release()
    elif args.mode in ["webcam"]:
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        while ret:
            try:
                frame = process_img(frame, face_detection)
                cv2.imshow('frame', frame)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            except Exception as e:
                print("No face detected in this frame.")
                # Handle the case when no face is detected
                # For example, you can skip this frame or perform alternative processing
            ret, frame = cap.read()
        cap.release()
        cv2.destroyAllWindows()

    #print(out.detections)